<a href="https://colab.research.google.com/github/IsaacFigNewton/CSC-480/blob/main/Revised_Roomba_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config

In [26]:
import networkx as nx
import json
import random
import numpy as np
import heapq as hq

contents = """7
5
_#a____
___#__#
*__#_*_
_______
____*__
"""

# save an offset map
offset_map = {
    # idx 0 => row offset
    # idx 1 => col offset
    "N": (-1, 0),
    "S": (1, 0),
    "E": (0, 1),
    "W": (0, -1)
}

## Helpers

## GameTree Class

In [27]:
# ensure the move is not OOB and not blocked by an obstacle
def is_legal_move(world, current_pos, proposed_pos):
  # check adjacency
  if not (proposed_pos[0] - current_pos[0], proposed_pos[1] - current_pos[1]) in offset_map.values():
    return False
  # check if the row idx is OOB
  if proposed_pos[0] < 0 or proposed_pos[0] >= world.num_rows:
    return False
  # check if the row idx is OOB
  if proposed_pos[1] < 0 or proposed_pos[1] >= world.num_cols:
    return False
  # check for obstacles
  if world.grid[proposed_pos[0]][proposed_pos[1]] == "#":
    return False
  return True

In [32]:
class GameTreeNode:
  def __init__(self, pos:tuple[int, int]):
    self.pos = pos
    # a set of child game states of the form:
    #   (<edge type (N, S, E, W)>, <new GameTreeNode>)
    self.children = set()

  def __hash__(self):
    return hash(self.pos)

  def get_proposed_move_pos(self, move):
    diff = offset_map[move]
    return (
        self.pos[0] + diff[0],
        self.pos[1] + diff[1]
    )

  def expand_children(self):
    # create a list of possible moves
    possible_moves = {
        move: self.get_proposed_move_pos(move)
        for move in offset_map.keys()
    }
    # filter it down to a list of legal moves
    for move, new_pos in possible_moves.items():
      if is_legal_move(world, self.pos, new_pos):
        self.children.add(Action(move, GameTreeNode(new_pos)))


class Action:
  def __init__(self, act_type:str, end_state:GameTreeNode):
    self.act_type = act_type
    self.end_state = end_state

  def __hash__(self):
    return hash(self.end_state.pos)

  def __lt__(self, other):
      return self.end_state.pos < other.end_state.pos

  def __gt__(self, other):
      return self.end_state.pos > other.end_state.pos


class Heuristic:
  def __init__(self, current_pos, function):
    self.pos = current_pos
    self.function = function

  def __call__(self, goal_pos):
    return self.function(self.current_pos, goal_pos)


class DecisionProcess:
  def __init__(self,
               act_store_type:str="stack"):
    if act_store_type == "stack":
      self.act_store = list()
    elif act_store_type == "priority queue":
      self.act_store = list()
      hq.heapify(self.act_store)

    self.act_store_type = act_store_type

  def push(self, action:Action, heuristic):
    match self.act_store_type:
      case "stack":
        self.act_store.append(action)
      case "priority queue":
        hq.heappush(
            self.act_store,
            (heuristic(action.end_state.pos), action)
        )

  def pop(self):
    match self.act_store_type:
      case "stack":
        return self.act_store.pop()
      case "priority queue":
        return hq.heappop(self.act_store)[1]

  def remove(self, action:Action):
    match self.act_store_type:
      case "stack":
        self.act_store.pop(action)
      case "priority queue":
        self.act_store = hq.heapify([
            a for a in self.act_store
            if a[1] != action
        ])

In [33]:
class World:
  def __init__(self, file_contents:str):
    world_lst = file_contents.split("\n")
    self.num_cols = int(world_lst[0])
    self.num_rows = int(world_lst[1])

    self.grid = list()
    self.dirty_cells = set()
    for i, row in enumerate(world_lst[2:-1]):
      self.grid.append(list())
      for j, c in enumerate(row):
        self.grid[i].append(c)
        if c == "*":
          self.dirty_cells.add((i, j))
    print("Successfully loaded world:")
    for row in self.grid:
      print(row)

    self.heuristic_map = {
        "DFS": None,
        "UCS": lambda start, end: (end[0]-start[0], end[1]-start[1])
    }

    self.act_store_map = {
        "DFS": DecisionProcess(
            act_store_type="stack"
        ),
        "UCS": DecisionProcess(
            act_store_type="priority queue"
        )
    }
    print(self.act_store_map["UCS"].act_store)


  def get_bot_pos_from_grid(self):
    for row in range(self.num_rows):
      for col in range(self.num_cols):
        if self.grid[row][col] == "a":
          return (row, col)
    raise ValueError("No bot found in the grid")


  # depth-first search
  def search(self, algorithm:str = "DFS"):
    agent = Agent(
        world=world,
        decision_process=self.act_store_map[algorithm],
        heuristic=self.heuristic_map[algorithm]
    )

    while self.dirty_cells:
      agent.select_action()
      # print([node[1].pos for node in agent.stack])
      # print(agent.world.dirt_count)
    print(f"{agent.nodes_generated} nodes generated.")
    print(f"{agent.nodes_expanded} nodes expanded.")



class Agent:
  def __init__(self,
               world: World,
               decision_process: DecisionProcess,
               heuristic=None):
    # vars for tracking position within game tree for DFS and UCS
    self.world = world

    self.current_path = list()
    self.pos = self.world.get_bot_pos_from_grid()
    # curry the heuristic with the
    self.heuristic = lambda x: heuristic(self.pos, x) if heuristic else None
    self.dp = decision_process

    self.model = GameTreeNode(self.pos)
    self.nodes_generated = 0
    self.nodes_expanded = 0


  def select_action(self):
      # if the bot is on dirt, clean it
      if self.world.grid[self.pos[0]][self.pos[1]] == "*":
          print("V")
          self.world.dirty_cells.remove(self.pos)
          self.world.grid[self.pos[0]][self.pos[1]] = "_"
          return

      # add the current node to the path
      self.current_path.append(self.pos)

      # expand all the current node's children
      self.model.expand_children()
      self.nodes_expanded += 1
      self.nodes_generated += len(self.model.children)
      for child in self.model.children:
        # add new, unexplored children to the set under consideration
        if child.end_state.pos not in self.current_path:
          self.dp.push(child, self.heuristic)

      # take the first available new move
      next_move = self.dp.pop()
      print(next_move.act_type)
      # print(next_move[0], next_move[1].pos)
      self.pos = next_move.end_state.pos
      self.model = next_move.end_state

# Test DFS

In [38]:
world = World(contents)

Successfully loaded world:
['_', '#', 'a', '_', '_', '_', '_']
['_', '_', '_', '#', '_', '_', '#']
['*', '_', '_', '#', '_', '*', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '*', '_', '_']
[]


In [39]:
world.search("DFS")

E
E
E
S
S
V
S
E
N
S
W
W
V
W
W
W
W
N
N
V
48 nodes generated.
17 nodes expanded.


In [36]:
for r in world.grid:
  print(r)

['_', '#', 'a', '_', '_', '_', '_']
['_', '_', '_', '#', '_', '_', '#']
['_', '_', '_', '#', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_']
